In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import json
from bs4 import BeautifulSoup

from sklearn.cluster import KMeans

# !conda install -c conda-forge geopy --yes 
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
!pip install folium
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 102kB 10.2MB/s ta 0:00:01
Folium installed
Libraries imported.


In [2]:
link_dataset = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
link_page = requests.get(link_dataset).text
soup = BeautifulSoup(link_page,'xml')

In [3]:
table_raw = soup.find("table")
data = {}
table_data = table_raw.tbody.find_all("tr")

for table in table_raw:
    t_headers = []
    for th in table_raw.find_all("th"):
        t_headers.append(th.text.replace('\n', ' ').strip())
    
    table_data_dt = []
    for tr in table_raw.tbody.find_all("tr"):
        t_row = {}
        
        for td, th in zip(tr.find_all("td"), t_headers): 
            t_row[th] = td.text.replace('\n', '').strip()
        table_data_dt.append(t_row)

data = table_data_dt[1:]

In [4]:
df = pd.DataFrame(data)
df = df[["Postal Code","Borough","Neighborhood"]]
df = df[df.Borough != "Not assigned"]
df = df.reset_index(drop = True)
df['Neighborhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df_toronto = pd.merge(df, df_geo, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
df_toronto.drop("Postal Code", axis=1, inplace=True)
df_toronto.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [7]:
downtown_toronto_df = df_toronto[df_toronto.Borough == "Downtown Toronto"].reset_index(drop=True)
downtown_toronto_df.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [8]:
CLIENT_ID = 'LIWVSK0FCTW4USDZYBGUB3N2JEJ2EACGFWESMYGAFJXSNNVC'
CLIENT_SECRET = 'VLXD0XQUOIABTJEYXEO4CHDSNFKVVWESTBWFWS52XWXD5OJW'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LIWVSK0FCTW4USDZYBGUB3N2JEJ2EACGFWESMYGAFJXSNNVC
CLIENT_SECRET:VLXD0XQUOIABTJEYXEO4CHDSNFKVVWESTBWFWS52XWXD5OJW


In [9]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [10]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [11]:
neighborhoods_data = newyork_data['features']

In [12]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [13]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [14]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [15]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="https://www.courseracapstoneproject.com")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [16]:
# Let's get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="https://www.courseracapstoneproject.com")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [17]:
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_df['Latitude'], downtown_toronto_df['Longitude'], downtown_toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [18]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_df['Latitude'], downtown_toronto_df['Longitude'], downtown_toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

In [19]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [20]:
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

grouping = plugins.MarkerCluster().add_to(map_manhattan)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_manhattan

In [21]:
def getNearbyVenues(names, latitudes,longitudes, radius=500, limit = 200):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_df['Neighborhood'],
                                   latitudes=downtown_toronto_df['Latitude'],
                                   longitudes=downtown_toronto_df['Longitude'],
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [23]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(1227, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [24]:
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,67,67,67,67,67,67
Christie,17,17,17,17,17,17
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [25]:
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [26]:
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [28]:
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2          Café  0.04
3      Pharmacy  0.04
4   Cheese Shop  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.21
1  Airport Terminal  0.14
2     Boat or Ferry  0.07
3           Airport  0.07
4  Sculpture Garden  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.06
2                Café  0.04
3      Sandwich Place  0.04
4                 Bar  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3     Restaurant  0.06
4    Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.10
1  Japanese Restaurant  0.05
2     Sushi Restaurant  0.05
3              Gay Bar  0.04
4           Restaurant  0.04


----Commerce C

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Cheese Shop,Pharmacy,Café,Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Coffee Shop,Airport,Airport Food Court,Airport Lounge,Boat or Ferry,Bar,Harbor / Marina,Sculpture Garden
2,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Thai Restaurant,Bubble Tea Shop,Burger Joint,Bar,Salad Place,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Men's Store,Pizza Place,Pub,Hotel
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Deli / Bodega
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Restaurant,Gym,Japanese Restaurant,American Restaurant,Seafood Restaurant,Asian Restaurant,Steakhouse
7,"Garden District, Ryerson",Clothing Store,Coffee Shop,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Hotel,Café,Japanese Restaurant,Lingerie Store,Electronics Store
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Restaurant,Brewery,Sporting Goods Shop,Fried Chicken Joint,Scenic Lookout,Pizza Place
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Bar,Grocery Store,Bakery,Dessert Shop,Park,Pizza Place


In [31]:
kclusters = 7

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 0, 6, 0, 0, 0, 0, 0, 0], dtype=int32)

In [32]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = downtown_toronto_df

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Theater,Café,Breakfast Spot,Restaurant,Ice Cream Shop,Beer Store
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Diner,Yoga Studio,Arts & Crafts Store,Bar,Bank,Mexican Restaurant,Beer Bar,Italian Restaurant,Hobby Shop
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Hotel,Café,Japanese Restaurant,Lingerie Store,Electronics Store
3,Downtown Toronto,St. James Town,43.651494,-79.375418,6,Restaurant,Coffee Shop,Café,Clothing Store,American Restaurant,Cocktail Bar,Cosmetics Shop,Gastropub,Japanese Restaurant,Seafood Restaurant
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Cheese Shop,Pharmacy,Café,Restaurant


In [33]:
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Theater,Café,Breakfast Spot,Restaurant,Ice Cream Shop,Beer Store
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Italian Restaurant,Cosmetics Shop,Bubble Tea Shop,Hotel,Café,Japanese Restaurant,Lingerie Store,Electronics Store
4,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Cheese Shop,Pharmacy,Café,Restaurant
5,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Thai Restaurant,Bubble Tea Shop,Burger Joint,Bar,Salad Place,Japanese Restaurant
6,Christie,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub
7,"Richmond, Adelaide, King",Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Thai Restaurant,Deli / Bodega,Gym,Cosmetics Shop,Salad Place
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Restaurant,Brewery,Sporting Goods Shop,Fried Chicken Joint,Scenic Lookout,Pizza Place
9,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Salad Place,American Restaurant,Seafood Restaurant,Sushi Restaurant
12,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Bar,Grocery Store,Bakery,Dessert Shop,Park,Pizza Place
14,Rosedale,Park,Trail,Playground,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [35]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Coffee Shop,Airport,Airport Food Court,Airport Lounge,Boat or Ferry,Bar,Harbor / Marina,Sculpture Garden


In [36]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Yoga Studio,Arts & Crafts Store,Bar,Bank,Mexican Restaurant,Beer Bar,Italian Restaurant,Hobby Shop


In [37]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Deli / Bodega


In [38]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"University of Toronto, Harbord",Café,Restaurant,Sandwich Place,Bookstore,Japanese Restaurant,Bar,Bakery,Sushi Restaurant,Comfort Food Restaurant,Beer Store


In [39]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 5, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Men's Store,Pizza Place,Pub,Hotel


In [40]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 6, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,St. James Town,Restaurant,Coffee Shop,Café,Clothing Store,American Restaurant,Cocktail Bar,Cosmetics Shop,Gastropub,Japanese Restaurant,Seafood Restaurant


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [43]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,69,69,69,69,69,69
Carnegie Hill,94,94,94,94,94,94
Central Harlem,48,48,48,48,48,48
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,44,44,44,44,44,44
East Village,100,100,100,100,100,100


In [44]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

In [46]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.10
1          Hotel  0.07
2            Gym  0.06
3    Coffee Shop  0.06
4  Memorial Site  0.04


----Carnegie Hill----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2  Pizza Place  0.04
3          Bar  0.04
4  Yoga Studio  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.06
1   Chinese Restaurant  0.06
2    French Restaurant  0.04
3  Fried Chicken Joint  0.04
4  American Restaurant  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.09
1          Art Gallery  0.06
2  American Restaurant  0.04
3    French Restaurant  0.03
4   Seafood Restaurant  0.03


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.09
1        Cocktail Bar  0.05
2              Bakery  0.05
3        Dessert Shop  0.03
4   Hotpot Restaurant  0.03


----Civic Center----
                  venue  freq
0           Coffee Shop  0.06
1  Gym / Fitness Center 

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Plaza,Gourmet Shop,Shopping Mall,Sandwich Place,BBQ Joint
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Bar,Bakery,Gym / Fitness Center,Gym,Italian Restaurant,Bookstore,Yoga Studio
2,Central Harlem,Chinese Restaurant,African Restaurant,Seafood Restaurant,American Restaurant,Bar,French Restaurant,Cosmetics Shop,Fried Chicken Joint,Dessert Shop,Cocktail Bar
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,French Restaurant,Bakery,Seafood Restaurant,Wine Shop,Cycle Studio,Café,Pizza Place
4,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Salon / Barbershop,Vietnamese Restaurant,Spa,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant,Optical Shop
5,Civic Center,Coffee Shop,Hotel,Gym / Fitness Center,Spa,French Restaurant,Cocktail Bar,Italian Restaurant,Bakery,Yoga Studio,Park
6,Clinton,Italian Restaurant,Theater,Gym / Fitness Center,Wine Shop,American Restaurant,Coffee Shop,Sandwich Place,Gym,Cocktail Bar,Hotel
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Pharmacy,Latin American Restaurant,Deli / Bodega,Park,Sandwich Place,Spa,Gas Station
8,East Village,Bar,Ice Cream Shop,Pizza Place,Mexican Restaurant,Cocktail Bar,Coffee Shop,Korean Restaurant,Wine Bar,Italian Restaurant,Dessert Shop
9,Financial District,Coffee Shop,Cocktail Bar,Pizza Place,Italian Restaurant,Gym,Gym / Fitness Center,Bar,Hotel,Salad Place,American Restaurant


In [49]:
kclusters = 7

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 6, 6, 6, 0, 4, 4, 5, 0, 6], dtype=int32)

In [50]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,6,Gym,Sandwich Place,Coffee Shop,Yoga Studio,Pharmacy,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place,Deli / Bodega
1,Manhattan,Chinatown,40.715618,-73.994279,6,Chinese Restaurant,Cocktail Bar,Bakery,Salon / Barbershop,Vietnamese Restaurant,Spa,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant,Optical Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,6,Café,Bakery,Mobile Phone Shop,Deli / Bodega,Chinese Restaurant,Mexican Restaurant,Donut Shop,Spanish Restaurant,Italian Restaurant,Latin American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,6,Mexican Restaurant,Lounge,Café,Restaurant,Park,Frozen Yogurt Shop,Bakery,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sandwich Place,Sushi Restaurant,Bakery,Cocktail Bar,Yoga Studio


In [51]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Pizza Place,Café,Coffee Shop,Deli / Bodega,Mexican Restaurant,Sandwich Place,Sushi Restaurant,Bakery,Cocktail Bar,Yoga Studio
8,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center,Yoga Studio,Juice Bar,French Restaurant,Exhibit,Spa,Pizza Place
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Café,American Restaurant,Ice Cream Shop,Clothing Store,Indian Restaurant,Bubble Tea Shop,Cosmetics Shop,Coffee Shop
19,East Village,Bar,Ice Cream Shop,Pizza Place,Mexican Restaurant,Cocktail Bar,Coffee Shop,Korean Restaurant,Wine Bar,Italian Restaurant,Dessert Shop
37,Stuyvesant Town,Park,Bar,Pet Service,Baseball Field,Gym / Fitness Center,Cocktail Bar,Harbor / Marina,Coffee Shop,Skating Rink,Heliport


In [53]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Coffee Shop,Café,Pizza Place,Bar,Bakery,Gym / Fitness Center,Gym,Italian Restaurant,Bookstore,Yoga Studio


In [54]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Upper West Side,Italian Restaurant,Coffee Shop,Bar,Wine Bar,Dessert Shop,Bakery,Ice Cream Shop,Bookstore,Pizza Place,Middle Eastern Restaurant
16,Murray Hill,Sandwich Place,Japanese Restaurant,Coffee Shop,Burger Joint,Italian Restaurant,American Restaurant,Bar,Pizza Place,Hotel,Gym / Fitness Center
21,Tribeca,Park,Italian Restaurant,American Restaurant,Wine Bar,Spa,Coffee Shop,Café,Men's Store,Poke Place,Bakery
27,Gramercy,Bar,American Restaurant,Coffee Shop,Bagel Shop,Italian Restaurant,Diner,Cocktail Bar,Ice Cream Shop,Mexican Restaurant,Thai Restaurant
29,Financial District,Coffee Shop,Cocktail Bar,Pizza Place,Italian Restaurant,Gym,Gym / Fitness Center,Bar,Hotel,Salad Place,American Restaurant
34,Sutton Place,Italian Restaurant,Park,Gym / Fitness Center,Gym,Coffee Shop,Furniture / Home Store,Hotel,Beer Bar,Department Store,Beer Garden
35,Turtle Bay,Italian Restaurant,Coffee Shop,Sushi Restaurant,Wine Bar,Park,Seafood Restaurant,Deli / Bodega,French Restaurant,Hotel,Café
36,Tudor City,Café,Park,Mexican Restaurant,Deli / Bodega,Sushi Restaurant,Greek Restaurant,Pizza Place,Spanish Restaurant,Dog Run,Coffee Shop


In [55]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Battery Park City,Park,Hotel,Gym,Coffee Shop,Memorial Site,Plaza,Gourmet Shop,Shopping Mall,Sandwich Place,BBQ Joint


In [56]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Italian Restaurant,Deli / Bodega,Mexican Restaurant,Sushi Restaurant,Seafood Restaurant,Climbing Gym,Bar,Fried Chicken Joint,Scenic Lookout
6,Central Harlem,Chinese Restaurant,African Restaurant,Seafood Restaurant,American Restaurant,Bar,French Restaurant,Cosmetics Shop,Fried Chicken Joint,Dessert Shop,Cocktail Bar
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cocktail Bar,Café,Deli / Bodega,Burger Joint,Gym,Gym / Fitness Center
14,Clinton,Italian Restaurant,Theater,Gym / Fitness Center,Wine Shop,American Restaurant,Coffee Shop,Sandwich Place,Gym,Cocktail Bar,Hotel
17,Chelsea,Coffee Shop,Art Gallery,American Restaurant,French Restaurant,Bakery,Seafood Restaurant,Wine Shop,Cycle Studio,Café,Pizza Place
31,Noho,Coffee Shop,Italian Restaurant,Mexican Restaurant,Sandwich Place,Pizza Place,Bookstore,Hotel,Grocery Store,Thai Restaurant,New American Restaurant
32,Civic Center,Coffee Shop,Hotel,Gym / Fitness Center,Spa,French Restaurant,Cocktail Bar,Italian Restaurant,Bakery,Yoga Studio,Park
38,Flatiron,Italian Restaurant,Gym / Fitness Center,New American Restaurant,Furniture / Home Store,Cycle Studio,Cosmetics Shop,Vegetarian / Vegan Restaurant,Coffee Shop,Yoga Studio,Salon / Barbershop


In [57]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 5, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Pharmacy,Latin American Restaurant,Deli / Bodega,Park,Sandwich Place,Spa,Gas Station
15,Midtown,Coffee Shop,Hotel,Clothing Store,Theater,Bakery,Steakhouse,Pizza Place,Bookstore,Japanese Restaurant,Gym


In [58]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 6, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Gym,Sandwich Place,Coffee Shop,Yoga Studio,Pharmacy,Steakhouse,Shopping Mall,Seafood Restaurant,Pizza Place,Deli / Bodega
1,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Salon / Barbershop,Vietnamese Restaurant,Spa,Bubble Tea Shop,Ice Cream Shop,Hotpot Restaurant,Optical Shop
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Deli / Bodega,Chinese Restaurant,Mexican Restaurant,Donut Shop,Spanish Restaurant,Italian Restaurant,Latin American Restaurant
3,Inwood,Mexican Restaurant,Lounge,Café,Restaurant,Park,Frozen Yogurt Shop,Bakery,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant
9,Yorkville,Italian Restaurant,Bar,Gym,Coffee Shop,Deli / Bodega,Pizza Place,Mexican Restaurant,Wine Shop,Sushi Restaurant,Japanese Restaurant
11,Roosevelt Island,Park,Deli / Bodega,Soccer Field,School,Scenic Lookout,Noodle House,Sandwich Place,Liquor Store,Dry Cleaner,Baseball Field
13,Lincoln Square,Plaza,Café,Concert Hall,Performing Arts Venue,Theater,American Restaurant,Italian Restaurant,Gym / Fitness Center,Gym,Wine Shop
20,Lower East Side,Chinese Restaurant,Cocktail Bar,Japanese Restaurant,Bakery,Pizza Place,Ramen Restaurant,Art Gallery,Coffee Shop,Mediterranean Restaurant,French Restaurant
22,Little Italy,Bakery,Chinese Restaurant,Italian Restaurant,Café,Ice Cream Shop,Pizza Place,Coffee Shop,Cocktail Bar,Salon / Barbershop,Mediterranean Restaurant
23,Soho,Italian Restaurant,Coffee Shop,Clothing Store,Boutique,Mediterranean Restaurant,Bakery,Salon / Barbershop,Sandwich Place,Pizza Place,Sushi Restaurant
